In [363]:
# import pakcages
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.cross_validation import cross_val_score

%matplotlib inline

In [364]:
#import data
sample = pd.read_csv('bank.csv', sep = ";")
data = pd.read_csv('bank-additional-full.csv',sep=';')

In [365]:
# making sure data is imported correctly
sample.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [366]:
# making sure data is imported correctly
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no


Question 1: . Use the file **bank.csv** to explore the dataset. Observe the features: Are they numbers? Are they strings? Are they binary? Are they continuous?

In [367]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4521 entries, 0 to 4520
Data columns (total 17 columns):
age          4521 non-null int64
job          4521 non-null object
marital      4521 non-null object
education    4521 non-null object
default      4521 non-null object
balance      4521 non-null int64
housing      4521 non-null object
loan         4521 non-null object
contact      4521 non-null object
day          4521 non-null int64
month        4521 non-null object
duration     4521 non-null int64
campaign     4521 non-null int64
pdays        4521 non-null int64
previous     4521 non-null int64
poutcome     4521 non-null object
y            4521 non-null object
dtypes: int64(7), object(10)
memory usage: 635.8+ KB


In [368]:
for col in sample.columns:
    print col
    print sample[col].unique()

age
[30 33 35 59 36 39 41 43 20 31 40 56 37 25 38 42 44 26 55 67 53 68 32 49 78
 23 52 34 61 45 48 57 54 63 51 29 50 27 60 28 21 58 22 46 24 77 75 47 70 65
 64 62 66 19 81 83 80 71 72 69 79 73 86 74 76 87 84]
job
['unemployed' 'services' 'management' 'blue-collar' 'self-employed'
 'technician' 'entrepreneur' 'admin.' 'student' 'housemaid' 'retired'
 'unknown']
marital
['married' 'single' 'divorced']
education
['primary' 'secondary' 'tertiary' 'unknown']
default
['no' 'yes']
balance
[ 1787  4789  1350 ...,  -333 -3313  1137]
housing
['no' 'yes']
loan
['no' 'yes']
contact
['cellular' 'unknown' 'telephone']
day
[19 11 16  3  5 23 14  6 17 20 13 30 29 27  7 18 12 21 26 22  2  4 15  8 28
  9  1 10 31 25 24]
month
['oct' 'may' 'apr' 'jun' 'feb' 'aug' 'jan' 'jul' 'nov' 'sep' 'mar' 'dec']
duration
[  79  220  185  199  226  141  341  151   57  313  273  113  328  261   89
  189  239  114  250  148   96  140  109  125  169  182  247  119  149   74
  897   81   40  958  354  150   97  132  765  

Question 2: transform the features to numerical features

In [369]:
#need transform categorical strings into numerical values
col_list =['job','marital','education','default','housing','loan','contact','month','poutcome']
print col_list

['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']


In [370]:
# create an array of only categorical values 
sample_categorical_values = np.array(sample[col_list])
sample_categorical_values

array([['unemployed', 'married', 'primary', ..., 'cellular', 'oct',
        'unknown'],
       ['services', 'married', 'secondary', ..., 'cellular', 'may',
        'failure'],
       ['management', 'single', 'tertiary', ..., 'cellular', 'apr',
        'failure'],
       ..., 
       ['technician', 'married', 'secondary', ..., 'cellular', 'aug',
        'unknown'],
       ['blue-collar', 'married', 'secondary', ..., 'cellular', 'feb',
        'other'],
       ['entrepreneur', 'single', 'tertiary', ..., 'cellular', 'apr',
        'other']], dtype=object)

In [371]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
le = LabelEncoder()

# first convert strings to numerical values using LabelEncoder
sample_data = le.fit_transform(sample_categorical_values[:,0])
print sample_data

for i in range(1,sample_categorical_values.shape[1]):
        sample_data = np.column_stack((sample_data, le.fit_transform(sample_categorical_values[:,i])))

print sample_data

[10  7  4 ...,  9  1  2]
[[10  1  0 ...,  0 10  3]
 [ 7  1  1 ...,  0  8  0]
 [ 4  2  2 ...,  0  0  0]
 ..., 
 [ 9  1  1 ...,  0  1  3]
 [ 1  1  1 ...,  0  3  1]
 [ 2  2  2 ...,  0  0  1]]


In [372]:
#double check numerical values and convert to float
sample_data[:5,:]
sample_cat_data = sample_data.astype(float)

In [373]:
# Use OneHotEncoder to convert data 
enc = OneHotEncoder()
sample_enc = enc.fit_transform(sample_cat_data)
#print out n_values
enc.n_values_

array([12,  3,  4,  2,  2,  2,  3, 12,  4])

In [374]:
enc.feature_indices_ 

array([ 0, 12, 15, 19, 21, 23, 25, 28, 40, 44])

In [375]:
#create a list of columns to help create a DF from np array
cols = [col_list[i] + '_' + str(j) for i in range(0,len(col_list)) for j in range(0,enc.n_values_[i]) ]
print cols
sample_cat_data_df = pd.DataFrame(sample_enc.toarray(),columns=cols)

['job_0', 'job_1', 'job_2', 'job_3', 'job_4', 'job_5', 'job_6', 'job_7', 'job_8', 'job_9', 'job_10', 'job_11', 'marital_0', 'marital_1', 'marital_2', 'education_0', 'education_1', 'education_2', 'education_3', 'default_0', 'default_1', 'housing_0', 'housing_1', 'loan_0', 'loan_1', 'contact_0', 'contact_1', 'contact_2', 'month_0', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'poutcome_0', 'poutcome_1', 'poutcome_2', 'poutcome_3']


In [376]:
# insert the columns back to sample df
sample[cols] = sample_cat_data_df[cols]

In [377]:
sample.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,...,month_6,month_7,month_8,month_9,month_10,month_11,poutcome_0,poutcome_1,poutcome_2,poutcome_3
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,...,0,0,0,0,1,0,0,0,0,1
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,...,0,0,1,0,0,0,1,0,0,0
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,...,0,0,0,0,0,0,1,0,0,0
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,...,1,0,0,0,0,0,0,0,0,1
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,...,0,0,1,0,0,0,0,0,0,1


In [378]:
# lastly, convert y into numerical values
sample['y'] = sample.y.map({'yes':1,'no':0})
print sample.y

0       0
1       0
2       0
3       0
4       0
5       0
6       0
7       0
8       0
9       0
10      0
11      0
12      0
13      1
14      0
15      0
16      0
17      0
18      0
19      0
20      0
21      0
22      0
23      0
24      0
25      0
26      0
27      0
28      0
29      0
       ..
4491    0
4492    0
4493    0
4494    1
4495    0
4496    0
4497    0
4498    0
4499    0
4500    0
4501    0
4502    0
4503    1
4504    1
4505    1
4506    0
4507    0
4508    0
4509    0
4510    0
4511    1
4512    0
4513    0
4514    0
4515    0
4516    0
4517    0
4518    0
4519    0
4520    0
Name: y, dtype: int64


In [380]:
X = sample.drop(col_list, axis =1)
X = X.drop('y', axis = 1)
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4521 entries, 0 to 4520
Data columns (total 51 columns):
age            4521 non-null int64
balance        4521 non-null int64
day            4521 non-null int64
duration       4521 non-null int64
campaign       4521 non-null int64
pdays          4521 non-null int64
previous       4521 non-null int64
job_0          4521 non-null float64
job_1          4521 non-null float64
job_2          4521 non-null float64
job_3          4521 non-null float64
job_4          4521 non-null float64
job_5          4521 non-null float64
job_6          4521 non-null float64
job_7          4521 non-null float64
job_8          4521 non-null float64
job_9          4521 non-null float64
job_10         4521 non-null float64
job_11         4521 non-null float64
marital_0      4521 non-null float64
marital_1      4521 non-null float64
marital_2      4521 non-null float64
education_0    4521 non-null float64
education_1    4521 non-null float64
education_2    4521

In [381]:
X.columns

Index([u'age', u'balance', u'day', u'duration', u'campaign', u'pdays',
       u'previous', u'job_0', u'job_1', u'job_2', u'job_3', u'job_4', u'job_5',
       u'job_6', u'job_7', u'job_8', u'job_9', u'job_10', u'job_11',
       u'marital_0', u'marital_1', u'marital_2', u'education_0',
       u'education_1', u'education_2', u'education_3', u'default_0',
       u'default_1', u'housing_0', u'housing_1', u'loan_0', u'loan_1',
       u'contact_0', u'contact_1', u'contact_2', u'month_0', u'month_1',
       u'month_2', u'month_3', u'month_4', u'month_5', u'month_6', u'month_7',
       u'month_8', u'month_9', u'month_10', u'month_11', u'poutcome_0',
       u'poutcome_1', u'poutcome_2', u'poutcome_3'],
      dtype='object')

In [382]:
y = sample.y

Question 3: Build a decision tree model to predict whether a prospect will buy the product.

In [383]:
# fit a classification tree with max_depth=5 on all data
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(max_depth=5, random_state=1)
dtc.fit(X,y)


DecisionTreeClassifier(compute_importances=None, criterion='gini',
            max_depth=5, max_features=None, max_leaf_nodes=None,
            min_density=None, min_samples_leaf=1, min_samples_split=2,
            random_state=1, splitter='best')

 Question 4: Evaluate the accuracy of your decision tree model using cross validation.

In [384]:
from sklearn.cross_validation import cross_val_score

scores = cross_val_score(dtc, X, y, cv=10, scoring='accuracy')
print 'Decision tree accuracy: {}'.format(np.mean(scores))

Decision tree accuracy: 0.898256461349


In [385]:
# compute the feature importances
pd.DataFrame({'feature':feature_cols, 'importance':dtc.feature_importances_})

,feature,importance
0,age,0.008354
1,balance,0.005765
2,day,0.035276
3,duration,0.493317
4,campaign,0.008180
5,pdays,0.047047
6,previous,0.000000
7,job_0,0.000000
8,job_1,0.000000
9,job_2,0.000000


In [386]:
#accurage is much lower with additional features, how about random forest and boost model?
# use cross-validation for random forest
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

rfc = RandomForestClassifier(max_depth=5, n_estimators=10, random_state=1)
scores = cross_val_score(rfc, X, y, cv=10, scoring='accuracy')
rfc.fit(X, y)
print 'Random forest accuracy: {}'.format(np.mean(scores))

abc = AdaBoostClassifier(n_estimators=10, random_state=1)
scores = cross_val_score(abc, X, y, cv=10, scoring='accuracy')
print 'AdaBoost accuracy: {}'.format(np.mean(scores))
abc.fit(X, y)

Random forest accuracy: 0.888520482916
AdaBoost accuracy: 0.900466408799


AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=10, random_state=1)

Quesiton 5: Repeat the analysis and cross validation with the file **bank-additional-full.csv**. How does the performance of the model change (with the additional training examples and additional features)?

In [387]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41188 entries, 0 to 41187
Data columns (total 21 columns):
age               41188 non-null int64
job               41188 non-null object
marital           41188 non-null object
education         41188 non-null object
default           41188 non-null object
housing           41188 non-null object
loan              41188 non-null object
contact           41188 non-null object
month             41188 non-null object
day_of_week       41188 non-null object
duration          41188 non-null int64
campaign          41188 non-null int64
pdays             41188 non-null int64
previous          41188 non-null int64
poutcome          41188 non-null object
emp.var.rate      41188 non-null float64
cons.price.idx    41188 non-null float64
cons.conf.idx     41188 non-null float64
euribor3m         41188 non-null float64
nr.employed       41188 non-null float64
y                 41188 non-null object
dtypes: float64(5), int64(5), object(11)
memory usa

In [388]:
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no


In [389]:
for col in data.columns:
    print col
    print data[col].unique()

age
[56 57 37 40 45 59 41 24 25 29 35 54 46 50 39 30 55 49 34 52 58 32 38 44 42
 60 53 47 51 48 33 31 43 36 28 27 26 22 23 20 21 61 19 18 70 66 76 67 73 88
 95 77 68 75 63 80 62 65 72 82 64 71 69 78 85 79 83 81 74 17 87 91 86 98 94
 84 92 89]
job
['housemaid' 'services' 'admin.' 'blue-collar' 'technician' 'retired'
 'management' 'unemployed' 'self-employed' 'unknown' 'entrepreneur'
 'student']
marital
['married' 'single' 'divorced' 'unknown']
education
['basic.4y' 'high.school' 'basic.6y' 'basic.9y' 'professional.course'
 'unknown' 'university.degree' 'illiterate']
default
['no' 'unknown' 'yes']
housing
['no' 'yes' 'unknown']
loan
['no' 'yes' 'unknown']
contact
['telephone' 'cellular']
month
['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'mar' 'apr' 'sep']
day_of_week
['mon' 'tue' 'wed' 'thu' 'fri']
duration
[ 261  149  226 ..., 1246 1556 1868]
campaign
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 19 18 23 14 22 25 16 17 15 20 56 39
 35 42 28 26 27 32 21 24 29 31 30 41 37 40 33 34 43]
pdays
[99

In [390]:
#need transform categorical strings into numerical values
data_cat_col =['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']
print data_cat_col

['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']


In [391]:
# create an array of only categorical values 
data_categorical_values = np.array(data[data_cat_col])
data_categorical_values

array([['housemaid', 'married', 'basic.4y', ..., 'may', 'mon',
        'nonexistent'],
       ['services', 'married', 'high.school', ..., 'may', 'mon',
        'nonexistent'],
       ['services', 'married', 'high.school', ..., 'may', 'mon',
        'nonexistent'],
       ..., 
       ['retired', 'married', 'university.degree', ..., 'nov', 'fri',
        'nonexistent'],
       ['technician', 'married', 'professional.course', ..., 'nov', 'fri',
        'nonexistent'],
       ['retired', 'married', 'professional.course', ..., 'nov', 'fri',
        'failure']], dtype=object)

In [392]:
le = LabelEncoder()

# first convert strings to numerical values using LabelEncoder
data_le = le.fit_transform(data_categorical_values[:,0])
print data_le

for i in range(1,data_categorical_values.shape[1]):
        data_le = np.column_stack((data_le, le.fit_transform(data_categorical_values[:,i])))

print data_le

[3 7 7 ..., 5 9 5]
[[3 1 0 ..., 6 1 1]
 [7 1 3 ..., 6 1 1]
 [7 1 3 ..., 6 1 1]
 ..., 
 [5 1 6 ..., 7 0 1]
 [9 1 5 ..., 7 0 1]
 [5 1 5 ..., 7 0 0]]


In [393]:
data_cat = data_le.astype(float)
# Use OneHotEncoder to convert data 
enc = OneHotEncoder()
data_enc = enc.fit_transform(data_cat)
#print out n_values
enc.n_values_

array([12,  4,  8,  3,  3,  3,  2, 10,  5,  3])

In [394]:
#create a list of columns to help create a DF from np array
cols = [data_cat_col[i] + '_' + str(j) for i in range(0,len(data_cat_col)) for j in range(0,enc.n_values_[i]) ]
print cols
data_cat_df = pd.DataFrame(data_enc.toarray(),columns=cols)

['job_0', 'job_1', 'job_2', 'job_3', 'job_4', 'job_5', 'job_6', 'job_7', 'job_8', 'job_9', 'job_10', 'job_11', 'marital_0', 'marital_1', 'marital_2', 'marital_3', 'education_0', 'education_1', 'education_2', 'education_3', 'education_4', 'education_5', 'education_6', 'education_7', 'default_0', 'default_1', 'default_2', 'housing_0', 'housing_1', 'housing_2', 'loan_0', 'loan_1', 'loan_2', 'contact_0', 'contact_1', 'month_0', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'day_of_week_0', 'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'poutcome_0', 'poutcome_1', 'poutcome_2']


In [395]:
data_cat_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41188 entries, 0 to 41187
Data columns (total 53 columns):
job_0            41188 non-null float64
job_1            41188 non-null float64
job_2            41188 non-null float64
job_3            41188 non-null float64
job_4            41188 non-null float64
job_5            41188 non-null float64
job_6            41188 non-null float64
job_7            41188 non-null float64
job_8            41188 non-null float64
job_9            41188 non-null float64
job_10           41188 non-null float64
job_11           41188 non-null float64
marital_0        41188 non-null float64
marital_1        41188 non-null float64
marital_2        41188 non-null float64
marital_3        41188 non-null float64
education_0      41188 non-null float64
education_1      41188 non-null float64
education_2      41188 non-null float64
education_3      41188 non-null float64
education_4      41188 non-null float64
education_5      41188 non-null float64
education_6

In [396]:
# insert the columns back to sample df
data[cols] = data_cat_df[cols]

In [397]:
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,month_8,month_9,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,poutcome_0,poutcome_1,poutcome_2
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,0,0,0,1,0,0,0,0,1,0
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,0,0,0,1,0,0,0,0,1,0
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,0,0,0,1,0,0,0,0,1,0
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,0,0,0,1,0,0,0,0,1,0
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,0,0,0,1,0,0,0,0,1,0


In [398]:
# lastly, convert y into numerical values
data['y'] = data.y.map({'yes':1,'no':0})
print data.y

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
41158    1
41159    1
41160    1
41161    0
41162    0
41163    1
41164    1
41165    1
41166    1
41167    0
41168    0
41169    0
41170    0
41171    1
41172    1
41173    1
41174    1
41175    0
41176    0
41177    0
41178    1
41179    0
41180    0
41181    1
41182    0
41183    1
41184    0
41185    0
41186    1
41187    0
Name: y, dtype: int64


In [404]:
y = data['y'] 
y

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
41158    1
41159    1
41160    1
41161    0
41162    0
41163    1
41164    1
41165    1
41166    1
41167    0
41168    0
41169    0
41170    0
41171    1
41172    1
41173    1
41174    1
41175    0
41176    0
41177    0
41178    1
41179    0
41180    0
41181    1
41182    0
41183    1
41184    0
41185    0
41186    1
41187    0
Name: y, dtype: int64

In [408]:
X = data.drop(data_cat_col, axis =1)
X = X.drop('y', axis=1)

In [409]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41188 entries, 0 to 41187
Data columns (total 63 columns):
age               41188 non-null int64
duration          41188 non-null int64
campaign          41188 non-null int64
pdays             41188 non-null int64
previous          41188 non-null int64
emp.var.rate      41188 non-null float64
cons.price.idx    41188 non-null float64
cons.conf.idx     41188 non-null float64
euribor3m         41188 non-null float64
nr.employed       41188 non-null float64
job_0             41188 non-null float64
job_1             41188 non-null float64
job_2             41188 non-null float64
job_3             41188 non-null float64
job_4             41188 non-null float64
job_5             41188 non-null float64
job_6             41188 non-null float64
job_7             41188 non-null float64
job_8             41188 non-null float64
job_9             41188 non-null float64
job_10            41188 non-null float64
job_11            41188 non-null float64

In [410]:
# fit a classification tree with max_depth=4 on all data
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(max_depth=4, random_state=1)
dtc.fit(X,y)

DecisionTreeClassifier(compute_importances=None, criterion='gini',
            max_depth=4, max_features=None, max_leaf_nodes=None,
            min_density=None, min_samples_leaf=1, min_samples_split=2,
            random_state=1, splitter='best')

In [411]:
from sklearn.cross_validation import cross_val_score

scores = cross_val_score(dtc, X, y, cv=10, scoring='accuracy')
print 'Decision tree accuracy: {}'.format(np.mean(scores))

Decision tree accuracy: 0.76613723159


In [412]:
#accurage is much lower with additional features, how about random forest and boost model?
# use cross-validation for random forest
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

rfc = RandomForestClassifier(max_depth=5, n_estimators=10, random_state=1)
scores = cross_val_score(rfc, X, y, cv=10, scoring='accuracy')
rfc.fit(X, y)
print 'Random forest accuracy: {}'.format(np.mean(scores))

abc = AdaBoostClassifier(n_estimators=10, random_state=1)
scores = cross_val_score(abc, X, y, cv=10, scoring='accuracy')
print 'AdaBoost accuracy: {}'.format(np.mean(scores))
abc.fit(X, y)

Random forest accuracy: 0.766572845416
AdaBoost accuracy: 0.802384123327


AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=10, random_state=1)

In [ ]:
# it's important to do feature reduction upfront:)